In [4]:
%pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 435.7 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/250.9 kB 178.6 kB/s eta 0:00:02
   -------------- ------------------------ 92.2/250.9 kB 228.2 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 315.4 kB/s eta 0:00:01
   -------------------------------------  245.8/250.9 kB 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import requests
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine


In [11]:
# Connect to the MySQL database
conn = mysql.connector.connect(
    host='ymtaz.sa',       # e.g., 'localhost'
    user='root',   # e.g., 'root'
    password='/er-x1k/9A(QFQ//',
    database='ymtazsa_db_dev' # Name of your database
)


In [ ]:
df1 = pd.read_sql('select email from lawyers_old where deleted_at is null', conn)
df2 = pd.read_sql('select email from service_users_old where deleted_at is null', conn)
df_combined = pd.concat([df1, df2])
df_combined = df_combined.drop_duplicates()

df_combined.to_csv('old_users_emails.csv', index=False)


In [14]:
df1 = pd.read_sql('select phone from lawyers_old where phone_code = 966 and deleted_at is null', conn)
df2 = pd.read_sql('select mobil from service_users_old where phone_code = 966 and deleted_at is null', conn)
df2.rename(columns={'mobil': 'phone'}, inplace=True)
df_combined = pd.concat([df1, df2])
df_combined = df_combined.drop_duplicates()

df_combined.to_csv('old_users_phones.csv', index=False)


C:\Users\bosay\AppData\Local\Temp\ipykernel_38148\3591661415.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql('select phone from lawyers_old where phone_code = 966 and deleted_at is null', conn)
C:\Users\bosay\AppData\Local\Temp\ipykernel_38148\3591661415.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql('select mobil from service_users_old where phone_code = 966 and deleted_at is null', conn)


In [ ]:

newsletter_df = pd.read_sql("SELECT email from newsletter", conn)

# Fetch emails from the 'clients' table into a DataFrame
clients_df = pd.read_sql("SELECT email FROM service_users", conn)

# Fetch emails from the 'lawyers' table into a DataFrame
lawyers_df = pd.read_sql("SELECT email FROM lawyers", conn)

visitors_df = pd.read_sql("SELECT email FROM visitors", conn)


# Combine the two DataFrames
emails_df = pd.concat([clients_df, lawyers_df, visitors_df], ignore_index=True)
emails_df['email'] = emails_df['email'].str.strip()

emails_df = emails_df[~emails_df['email'].isin(newsletter_df['email'])]

# Remove duplicates if necessary
emails_df.drop_duplicates(inplace=True)



In [29]:
emails_df.count()
emails_df["email"] = emails_df["email"].astype(str)

In [30]:
engine = create_engine('mysql+mysqlconnector://root:/er-x1k/9A(QFQ//@ymtaz.sa/ymtazsa_db_dev')
emails_df.to_sql('newsletter', con=engine, if_exists='append', index=False)


1710

In [24]:
def get_countries():
    url = 'https://restcountries.com/v3.1/all'
    response = requests.get(url)
    countries = response.json()
    country_data = []
    for country in countries:
        if country['idd'] != {}:
            country_suffix = country['idd']['suffixes'][0]
            country_root = country['idd']['root']
            country_info = {
                'code': country['cca2'],
                'name_ar': country['translations']['ara']['common'],
                'phone_code': country_root + country_suffix,
            }
            country_data.append(country_info)
        else:
            country_suffix = ''
            country_root = ''
    
    return country_data

In [3]:
def get_geonames_data(country_code, username):
    url = f'http://api.geonames.org/countryInfoJSON?country={country_code}&username={username}'
    response = requests.get(url)
    geonames_info = response.json()
    
    if 'geonames' not in geonames_info or len(geonames_info['geonames']) == 0:
        return []
    
    country_geoname_id = geonames_info['geonames'][0]['geonameId']
    
    regions_url = f'http://api.geonames.org/childrenJSON?geonameId={country_geoname_id}&username={username}'
    regions_response = requests.get(regions_url)
    regions = regions_response.json().get('geonames', [])
    
    regions_data = []
    for region in regions:
        region_name_ar = region['toponymName']
        region_geoname_id = region['geonameId']
        
        cities_url = f'http://api.geonames.org/searchJSON?q=&featureCode=PPLA&featureCode=PPLC&adminCode1={region_geoname_id}&username={username}'
        cities_response = requests.get(cities_url)
        cities = cities_response.json().get('geonames', [])
        
        cities_ar = [city['toponymName'] for city in cities]
        
        regions_data.append({
            'region_name_ar': region_name_ar,
            'cities_ar': cities_ar
        })
    
    return regions_data

In [34]:
geonames_username = 'Chosey'
countries = get_countries()
countriesWithRegions = []
for country in countries:
        regions_and_cities = get_geonames_data(country['code'], geonames_username)
        country['regions'] = regions_and_cities
        countriesWithRegions.append(country)

{'common': 'Heard Island and McDonald Islands', 'official': 'Heard Island and McDonald Islands', 'nativeName': {'eng': {'official': 'Heard Island and McDonald Islands', 'common': 'Heard Island and McDonald Islands'}}}
{'common': 'Antarctica', 'official': 'Antarctica'}


In [35]:
countriesWithRegions

[{'code': 'WF',
  'name_ar': 'واليس وفوتونا',
  'phone_code': '+681',
  'regions': [{'region_name_ar': 'Alo', 'cities_ar': []},
   {'region_name_ar': 'Sigave', 'cities_ar': []},
   {'region_name_ar': 'Uvéa', 'cities_ar': []}]},
 {'code': 'IS',
  'name_ar': 'آيسلندا',
  'phone_code': '+354',
  'regions': [{'region_name_ar': 'Höfuðborgarsvæði', 'cities_ar': []},
   {'region_name_ar': 'Austurland', 'cities_ar': []},
   {'region_name_ar': 'Norðurland Eystra', 'cities_ar': []},
   {'region_name_ar': 'Norðurland Vestra', 'cities_ar': []},
   {'region_name_ar': 'Suðurland', 'cities_ar': []},
   {'region_name_ar': 'Suðurnes', 'cities_ar': []},
   {'region_name_ar': 'Vesturland', 'cities_ar': []},
   {'region_name_ar': 'Vestfirðir', 'cities_ar': []}]},
 {'code': 'LU',
  'name_ar': 'لوكسمبورغ',
  'phone_code': '+352',
  'regions': [{'region_name_ar': 'Capellen', 'cities_ar': []},
   {'region_name_ar': 'Clervaux', 'cities_ar': []},
   {'region_name_ar': 'Diekirch', 'cities_ar': []},
   {'region_n

In [1]:
import random
import string
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(
    host='ymtaz.sa',       # e.g., 'localhost'
    user='root',   # e.g., 'root'
    password='/er-x1k/9A(QFQ//',
    database='ymtazsa_db_dev' # Name of your database
)
lawyers = pd.read_sql("SELECT l.* FROM service_users AS l LEFT JOIN referral_codes AS rc ON rc.client_id = l.id WHERE rc.client_id IS NULL;", conn)

def generate_referral_code(length=8):
    characters = string.ascii_uppercase + string.digits
    referral_code = ''.join(random.choice(characters) for _ in range(length))
    return referral_code

for index, row in lawyers.iterrows():
    referral_code = generate_referral_code()
    lawyer_id = row['id']
    
    # Update the database with the new referral code
    update_query = "INSERT INTO referral_codes (client_id, referral_code) VALUES (%s, %s)"
    cursor = conn.cursor()
    cursor.execute(update_query,  (lawyer_id, referral_code))
    conn.commit()

# Close the connection
cursor.close()
conn.close()



C:\Users\bosay\AppData\Local\Temp\ipykernel_27964\4110728187.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lawyers = pd.read_sql("SELECT l.* FROM service_users AS l LEFT JOIN referral_codes AS rc ON rc.client_id = l.id WHERE rc.client_id IS NULL;", conn)
